# 第8课：支持向量机 (SVM)

## 学习目标
- 理解支持向量机的核心思想
- 掌握线性 SVM 和核 SVM
- 学习 SVM 用于分类和回归
- 了解核函数的作用

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

## 1. SVM 基本概念

支持向量机 (Support Vector Machine) 是一种强大的监督学习算法，核心思想是：

- **最大间隔**：找到一个超平面，使得离它最近的样本点（支持向量）到它的距离最大
- **支持向量**：离决策边界最近的那些样本点
- **核技巧**：通过核函数将数据映射到高维空间，解决非线性问题

## 2. 线性 SVM

In [ ]:
# 生成线性可分数据
np.random.seed(42)
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, n_clusters_per_class=1,
                           class_sep=2.0, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")

In [ ]:
# 训练线性 SVM
linear_svm = SVC(kernel='linear', C=1.0, random_state=42)
linear_svm.fit(X_train_scaled, y_train)

# 预测
y_pred = linear_svm.predict(X_test_scaled)
print(f"准确率: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n支持向量数量: {linear_svm.n_support_}")

In [ ]:
# 可视化决策边界
def plot_svm_decision_boundary(model, X, y, title):
    plt.figure(figsize=(10, 6))
    
    # 创建网格
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # 预测
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # 绘制
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black')
    
    # 标记支持向量
    plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
                s=200, facecolors='none', edgecolors='green', linewidths=2,
                label='Support Vectors')
    
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.show()

plot_svm_decision_boundary(linear_svm, X_train_scaled, y_train, 'Linear SVM Decision Boundary')

## 3. 核 SVM（非线性）

当数据线性不可分时，使用核函数将数据映射到高维空间。

常用核函数：
- **linear**: 线性核，$K(x, y) = x^T y$
- **poly**: 多项式核，$K(x, y) = (\gamma x^T y + r)^d$
- **rbf**: 高斯径向基核（默认），$K(x, y) = \exp(-\gamma ||x-y||^2)$
- **sigmoid**: Sigmoid 核

In [ ]:
# 生成非线性数据 - 月牙形
X_moons, y_moons = make_moons(n_samples=300, noise=0.15, random_state=42)

# 生成非线性数据 - 同心圆
X_circles, y_circles = make_circles(n_samples=300, noise=0.1, factor=0.5, random_state=42)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='RdYlBu', edgecolors='black')
axes[0].set_title('Moons Dataset')

axes[1].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='RdYlBu', edgecolors='black')
axes[1].set_title('Circles Dataset')

plt.tight_layout()
plt.show()

In [ ]:
# 比较不同核函数
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, kernel in enumerate(kernels):
    # 月牙数据
    svm_moons = SVC(kernel=kernel, gamma='auto', random_state=42)
    svm_moons.fit(X_moons, y_moons)
    
    # 同心圆数据
    svm_circles = SVC(kernel=kernel, gamma='auto', random_state=42)
    svm_circles.fit(X_circles, y_circles)
    
    # 绘制月牙
    ax = axes[0, idx]
    h = 0.02
    x_min, x_max = X_moons[:, 0].min() - 0.5, X_moons[:, 0].max() + 0.5
    y_min, y_max = X_moons[:, 1].min() - 0.5, X_moons[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svm_moons.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='RdYlBu', edgecolors='black', s=20)
    ax.set_title(f'Moons - {kernel}')
    
    # 绘制同心圆
    ax = axes[1, idx]
    x_min, x_max = X_circles[:, 0].min() - 0.5, X_circles[:, 0].max() + 0.5
    y_min, y_max = X_circles[:, 1].min() - 0.5, X_circles[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svm_circles.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='RdYlBu', edgecolors='black', s=20)
    ax.set_title(f'Circles - {kernel}')

plt.tight_layout()
plt.show()

## 4. 重要参数

### C 参数（正则化）
- C 值大：更严格，允许更少的错误分类
- C 值小：更宽松，允许更多的错误分类（更好的泛化）

### gamma 参数（RBF 核）
- gamma 值大：决策边界更复杂，容易过拟合
- gamma 值小：决策边界更平滑，可能欠拟合

In [ ]:
# C 参数的影响
C_values = [0.01, 0.1, 1, 10, 100]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for idx, C in enumerate(C_values):
    svm = SVC(kernel='rbf', C=C, gamma='auto', random_state=42)
    svm.fit(X_moons, y_moons)
    
    ax = axes[idx]
    h = 0.02
    x_min, x_max = X_moons[:, 0].min() - 0.5, X_moons[:, 0].max() + 0.5
    y_min, y_max = X_moons[:, 1].min() - 0.5, X_moons[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='RdYlBu', edgecolors='black', s=20)
    ax.set_title(f'C = {C}')

plt.tight_layout()
plt.show()

In [ ]:
# gamma 参数的影响
gamma_values = [0.01, 0.1, 1, 10, 100]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for idx, gamma in enumerate(gamma_values):
    svm = SVC(kernel='rbf', C=1, gamma=gamma, random_state=42)
    svm.fit(X_moons, y_moons)
    
    ax = axes[idx]
    h = 0.02
    x_min, x_max = X_moons[:, 0].min() - 0.5, X_moons[:, 0].max() + 0.5
    y_min, y_max = X_moons[:, 1].min() - 0.5, X_moons[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='RdYlBu', edgecolors='black', s=20)
    ax.set_title(f'gamma = {gamma}')

plt.tight_layout()
plt.show()

## 5. 网格搜索调参

In [ ]:
# 划分数据
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42)

# 定义参数网格
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 10],
    'kernel': ['rbf', 'poly']
}

# 网格搜索
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_m, y_train_m)

print(f"最佳参数: {grid_search.best_params_}")
print(f"最佳交叉验证分数: {grid_search.best_score_:.4f}")

# 测试集评估
best_model = grid_search.best_estimator_
y_pred_m = best_model.predict(X_test_m)
print(f"测试集准确率: {accuracy_score(y_test_m, y_pred_m):.4f}")

## 6. 支持向量回归 (SVR)

In [ ]:
# 生成回归数据
np.random.seed(42)
X_reg = np.sort(5 * np.random.rand(100, 1), axis=0)
y_reg = np.sin(X_reg).ravel() + np.random.randn(100) * 0.1

# 训练不同核的 SVR
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel='linear', C=100)
svr_poly = SVR(kernel='poly', C=100, degree=3)

svr_rbf.fit(X_reg, y_reg)
svr_lin.fit(X_reg, y_reg)
svr_poly.fit(X_reg, y_reg)

# 预测
X_plot = np.linspace(0, 5, 100).reshape(-1, 1)

plt.figure(figsize=(12, 5))

plt.scatter(X_reg, y_reg, color='darkorange', label='Data')
plt.plot(X_plot, svr_rbf.predict(X_plot), color='navy', label='RBF')
plt.plot(X_plot, svr_lin.predict(X_plot), color='green', label='Linear')
plt.plot(X_plot, svr_poly.predict(X_plot), color='red', label='Polynomial')

plt.xlabel('X')
plt.ylabel('y')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

## 7. 多分类 SVM

In [ ]:
from sklearn.datasets import load_iris

# 加载鸢尾花数据
iris = load_iris()
X_iris, y_iris = iris.data, iris.target

# 划分数据
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=42)

# 标准化
scaler_iris = StandardScaler()
X_train_i_scaled = scaler_iris.fit_transform(X_train_i)
X_test_i_scaled = scaler_iris.transform(X_test_i)

# 训练 SVM (默认使用 OvO 策略)
svm_iris = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
svm_iris.fit(X_train_i_scaled, y_train_i)

# 预测
y_pred_i = svm_iris.predict(X_test_i_scaled)

print("多分类 SVM 结果:")
print(f"准确率: {accuracy_score(y_test_i, y_pred_i):.4f}")
print(f"\n分类报告:\n{classification_report(y_test_i, y_pred_i, target_names=iris.target_names)}")

In [ ]:
# 混淆矩阵
cm = confusion_matrix(y_test_i, y_pred_i)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Iris Classification')
plt.show()

## 8. 练习题

### 练习1：乳腺癌分类
使用 sklearn 的乳腺癌数据集，训练一个 SVM 分类器

In [ ]:
from sklearn.datasets import load_breast_cancer

# 加载数据
cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target

# 在这里编写代码


### 练习2：参数调优
对上面的乳腺癌分类器进行网格搜索调参

In [ ]:
# 在这里编写代码


## 9. 本课小结

1. **核心思想**：SVM 寻找最大间隔的决策边界
2. **支持向量**：离边界最近的样本，决定了边界位置
3. **核函数**：将非线性问题转化为高维空间的线性问题
4. **常用核**：linear、poly、rbf（最常用）、sigmoid
5. **重要参数**：
   - C：正则化强度，控制错误分类的惩罚
   - gamma：RBF 核的带宽，影响决策边界复杂度
6. **SVR**：支持向量机的回归版本
7. **多分类**：默认使用 OvO (One-vs-One) 策略

### SVM 的优缺点

**优点**：
- 在高维空间表现良好
- 内存效率高（只存储支持向量）
- 通过核函数处理非线性问题

**缺点**：
- 大数据集训练较慢
- 对特征缩放敏感
- 参数调优较复杂